# Problema da Dieta aplicado ao planejamento de refeições da Marinha

Este *notebook* tem o intuito de explicar e exemplificar o trabalho "Problema da dieta: uma aplicação prática para o Navio Hidroceanográfico 'Taurus'".

## Estrutura do Notebook

Inicialmente será apresentado um breve resumo do trabalho original com foco no problema estudado. Na sequência o é proposta uma formulação e implementação do modelo proposto pelos autores. Inicialmente são apresentados os requisitos para a execução do código. Em seguida são descritos os dados necessários para o modelo. Por fim, a função objetivo e as restrições são explicadas. Após a explicação do modelo ser finalizada, uma célula permite a execução do exemplo implementado.

## Trabalho original

O problema proposto pelos autores é uma variação do problema da dieta que visa auxiliar no planejamento das refeições de uma viagem de 10 dias de soldados da marinha brasileira. O intuito é reduzir o custo de produção das refeições, além de suprir os requisitos nutricionais mínimos dos soldados.

Além das restrições tradicionais do problemada dieta, os autores consideram fatores culturais da alimentação brasileira e a diversificação de refeições. É considerado um cardápio com 91 possíveis pratos.

São consideradas três principais refeições a serem planejadas: desjejum, almoço e jantar. Para o desejejum é considerado uma refeição constante, ou seja, um custo e quantidade de nutrientes constantes. Logo, é necessário planejar somente o almoço e jantar, ou seja, 20 refeições.

Os autores consideram cada prato como um ingrediente e calculam os valores nutricionais dos pratos utilizando tabelas nutricionais da *U.S. Department of Agriculture, Agricultural Research Service. 2001* e da *Tabela de
Composição UNICAMP*. Os preços dos pratos são calculados com base nos valor dos ingredientes na época.

Inicialmente, a formulação clássica para o problema foi testada, mas, como esperado, a diversificação dos alimentos foi baixa e não levou em conta fatores culturais. 

No Brasil duas principais características culturais podem ser facilmente identificadas na alimentação. O primeiro deles são os pratos principais, geralmente feitos a base de carne. O segundo é a presença de pratos baseados em arroz e feijão. Para garantir que esses elementos fossem incluídos, restrições foram adicionadas ao problema da dieta para que um número mínimo de pratos principais fossem incluídos. Também foram adicionadas restrições para que fosse garantido que pratos baseados em arroz e fejão estivessem incluído no mínimo em todos os almoços (10 refeições) e no máximo em todas as refeições (20). Foram feitos testes com pratos principais considerando a presença deles no mínimo em 10 (todos os almoços), 15 (todos os almoços e metade dos jantares) ou 20 (todas as refeições) refeições.

Para garantir a diversificação, limitou-se o número de repetições de cada prato não cultural. Foram realizados testes com no máximo 10, 5, 4, 3 e 2 repetições.

Observou-se que o aumento de diversificação e a inclusão de pratos principais gerou aumento no custo, como pode ser visto pela tabela a seguir (extraída do trabalho original).


| | | Mínimo de pratos com carne (P) | |
|:----------------------|:---------|:--------:|---------:|
|Repetições de Refeição (O) | P >= 10 | P >= 15 | P >= 20 |
| O <= 10 | R\$ 28,50 | R\$ 31,95 | R\$ 35,65 |
| O <= 5  | R\$ 30,52 | R\$ 33,90 | R\$ 37,62 |
| O <= 4  | R\$ 31,46 | R\$ 34,85 | R\$ 39,22 |
| O <= 3  | R\$ 32,77 | R\$ 36,70 | R\$ 41,46 |
| O <= 2  | R\$ 36,24 | R\$ 40,21 | R\$ 45,18 |

Os diferentes resultados obtidos pelos autores permitem que a decisão de aumentar a variabilidade do cardápio assim como a adição de mais pratos principais ao cardápio seja do gestor do navio, tornando o modelo proposto uma ferramenta de auxílio na tomada de decisão.



### Referência do trabalho original

PESSÔA, Leonardo Antonio Monteiro; LINS, Marcos Pereira Estellita; TORRES, Nilson Trevisan. Problema da dieta: uma aplicação prática para o Navio Hidroceanográfico “Taurus”. 2009.

# Requisitos

É necessário o pacote [python-mip](https://www.python-mip.com/). Execute a célula abaixo para importar os pacotes utilizados.

In [ ]:
import os
from mip import *

### Solver

O python-mip permite a execução com dois dieferentes solvers: CBC ou Gurobi. Execute a primeira célula caso deseje executar utilizando o CBC e a segunda caso deseje utilizar o Gurobi.

In [ ]:
solver_code = "CBC"

In [ ]:
solver_code = "GRB"

# Modelo Matemático

O modelo matemático descrito no estudo é baseado na modelagem clássica do problema da dieta. O objetivo dos autores foi a minimização do custo de alimentação. As restrições são resumidas abaixo:

- Suprir as necessidades mínimas de determinados nutrientes
- Garantir que determinados pratos (pratos princiapais) estejam presentes em todas ou na maioria das refeições
- Garantir que alimentos que são parte da cultura alimentar estejam presentes em um número mínimo de refeições
- Garantir diversidade de refeições

A seguir será descrita a formulação matemática do problema. Nota-se que alguns detalhes são omitidos para simplificar a representação do problema, como por exemplo o custo fixado na função objetivo que é gerado a partir do café da manhã. Essa simplificação não implica em perda da generalidade.

A classe *DietSolver* implementa o modelo que será apresentado. Seu construtor, a função de otimização e funções de escrita são apresentadas abaixo.

In [ ]:
class DietSolver():

    # Initialize class object
    # input_data has the information about the dishes
    def __init__(self, input_data):
        self.dishes = input_data["dishes"]
        self.costs = input_data["costs"]
        self.components = input_data["components"]
        self.components_ingerdients_amount = input_data["components_ingerdients_amount"]
        self.components_min_amount = input_data["components_min_amount"]
        self.main_dishes = input_data["main_dishes"]
        self.cultural_dishes = input_data["cultural_dishes"]
        self.main_dish_min_rep = input_data["main_dish_min_rep"]
        self.cultural_dish_min_rep = input_data["cultural_dish_min_rep"]
        self.cultural_dish_max_rep = input_data["cultural_dish_max_rep"]
        self.others_max_rep = input_data["others_max_rep"]
        self.total_to_make = input_data["total_to_make"]

        # Create a Python-MIP model
        self.create_model()
        # Create a a variable for each dish
        self.create_variables()
        # Formulate the objective function
        self.create_objective_function()
        # Formulate each constraint
        self.create_constraints()
    
    # Create a Python-MIP model
    def create_model(self):
        self.model = Model(solver_name=solver_code, sense=MINIMIZE)

    def optimize(self):
        self.status = self.model.optimize()

    def print_model(self):
        self.model.write("temp.lp")
        with open("temp.lp", "r") as f_model:
            text = f_model.read()
            print(text)
            os.remove("temp.lp")

    def print_solution(self):
        if (self.status == OptimizationStatus.OPTIMAL):
            print("Solução ótima encontrada")
            print("Custo: " + str(self.model.objective_value))
            for i, var in enumerate(self.variables):
                print(
                    "Quantidade da refeição \"" 
                    + 
                    self.dishes[i] 
                    + 
                    "\": " 
                    + 
                    str(var.x)
                )
            return
        if(self.status == OptimizationStatus.INFEASIBLE):
            print("Modelo infactível")
            return
        
        if (self.status == OptimizationStatus.UNBOUNDED):
            print("Solução ilimitada (f.o. tende ao infinito)")
            return

## Dados do problema

- $N$: Número de possíveis pratos a serem preparadas para uma refeição.
    - $1 ... K^{arroz}$ são pratos baseados em arroz
    - $K^{arroz}+1 ... K^{feijão}$ são pratos baseados em feijão
    - $K+1 ... L$ são pratos principais, com $K = K^{feijao}$
    - $L+1 ... N$ são os demais pratos

- $M$: Número de componentes (vitaminas, sais minerais, etc) a serem considerados na refeição.

- $a_{ij}$: Quantidade do componente $i$ no prato $j$. Com $i = 1 ... M$ e $j = 1 ... N$.

- $b_{i}$: Quantidade mínima do componente $i$ que deve estar presente na refeição

- $c_{j}$: Custo do produto $j$, com $j = 1 ... N$

- $P$: Número mínimo de pratos principais a ser incluído no planejamento de refeições

- $C^{min}$, $C^{max}$: Número mínimo e máximo de repetições de pratos culturais

- $O^{max}$: Número máximo de repetições de pratos não culturais

- $H$: Número de refeições a serem preparadas

### Exemplo

Abaixo segue o código de um exemplo reduzido. Os valores utilizados são arbitrários. Como pode ser visto, algumas mudanças em código foram feitas para facilitar a identificação das constantes, contudo, o princípio da implementação é o mesmo.


In [ ]:
# List of all dishes (size = N)
dishes = [
    "Almôndegas",  
    "Strogonoff", 
    "Churrasco", 
    "Arroz", 
    "Feijão", 
    "Arroz à Grega", 
    "Lasanha",
    "Purê de Batata",
    "Sopa de Legumes",
    "Lentilhas",
    "Ervilhas",
    "Doce de Leite",
    "Doce de Abóbora",
    "Banana",
    "Maçã",
    "Leite"
]

# Costs array (c_j)
costs_per_person = [
    0.80, 
    0.75, 
    1.10,
    0.05,
    0.15, 
    0.10, 
    0.40, 
    0.20, 
    0.25, 
    0.15,
    0.15,
    0.5,
    0.3,
    0.1,
    0.15,
    0.40
]

# List of components (size = M)
components = [
    "proteina",
    "calcio",
    "ferro",
    "calorias"
]

# Amount of components in dishes (a_ij)
components_ingerdients_amount = [
    [10, 17, 26, 3, 8, 5, 8, 3, 1, 1, 5, 7, 1, 2, 0, 3],
    [4, 4, 5, 0, 0, 0, 4, 0, 0, 0, 0, 250, 13, 8, 6, 123],
    [2, 2, 3, 0, 5, 0, 0, 0, 0, 2, 2, 0, 1, 1, 0, 0],
    [190, 150, 230, 130, 350, 160, 130, 80, 40, 170, 80, 310, 200, 120, 50, 60]
]

# Min amount of components (b_i)
components_min_amount = [
    460,
    3000,
    80,
    25000
]

# List of main dishes (K+1 to L)
main_dishes = [
    "Almôndegas",  
    "Strogonoff", 
    "Churrasco"
]

# List of cultural dishes (1 to K)
cultural_dishes = [
    ["Arroz", "Arroz à Grega"],
    ["Feijão"]
]

# minimum amount of main dishes (P)
main_dish_min_rep = 20
# minimum of cultural dishes (C^{min})
cultural_dish_min_rep = 10
# maximum of cultural dishes (C^{max})
cultural_dish_max_rep = 20
# maximum of repetition of non-cultural dishes (O^{max})
others_max_rep = 10
# Number of meals (H)
total_to_make = 20


### Variáveis

- $x_{j}$: Quantidade a ser feita do prato $j$, $j = 1 ... N$.
    - $x_{1} ... x_{K}$ referem às quantidades dos pratos culturais
    - $x_{K+1} ... x_{L}$ referem às quantidades dos pratos principais
    - $x_{L+1} ... x_{N}$ referem às quantidades dos demais pratos

In [ ]:
class DietSolver(DietSolver):    
    # Create a a variable for each dish
    def create_variables(self):
        self.variables = [
            self.model.add_var(
                name="x_" + dish, 
                lb=0, 
                var_type=CONTINUOUS
            )
            for dish in self.dishes
        ]

### Função Objetivo

A função objetivo pode ser definida como a multiplicação dos custos dos pratos pela quantidade de cada prato:

$min$ $\sum_{j=1}^{N}{c_{j} * x_{j}}$

In [ ]:
class DietSolver(DietSolver):    
    # Formulate the objective function
    def create_objective_function(self):
        self.model.objective = xsum([
            self.costs[i] * self.variables[i]
            for i in range(len(self.costs))
        ])


### Restrições

As restrições são criadas pelo método *create_constraints*. A seguir, cada subconjunto de restrições será desrito.

In [ ]:
class DietSolver(DietSolver):  
    # Formulate each constraint
    def create_constraints(self):
        # Ensures that the minimum amount of each component is match
        components_min_amount = self.add_components_min_amount()
        
        # Ensure that the minumum amount of main dishes is match
        main_dish_rep_constraints = self.add_main_dish_rep_constraints()
        
        # Ensures that the cultural minimum dishes are match
        cultural_dish_min_rep_constraints = self.add_cultural_dish_min_rep_constraints()

        # Ensures that the cultural maximum dishes are match
        cultural_dish_max_rep_constraints = self.add_cultural_dish_max_rep_constraints()


        # For each dish that is not a cultural dish
        # Ensures that the maximum number of dishes is not exceeded
        other_dishes_max_rep = self.add_other_dishes_max_rep()

        # Store the constraints
        self.constraints = (
            components_min_amount 
            + 
            main_dish_rep_constraints
            +
            cultural_dish_min_rep_constraints
            +
            cultural_dish_max_rep_constraints
            +
            other_dishes_max_rep
        )

#### Quantidade mínima de componentes

As restrições abaixo definem que a quantidade mínima de cada componente deve estar contida na refeição.

$\sum_{j=1}^{N}{a_{ij}x_{j}} \leq b_{j}$, com $i = 1 ... M$

In [ ]:
class DietSolver(DietSolver):
    def add_components_min_amount(self):
        return [
            self.model.add_constr(
                xsum([
                    self.components_ingerdients_amount[i][j] 
                    * 
                    self.variables[j]
                    for j in range(len(self.dishes))
                ])
                >= 
                self.components_min_amount[i],

                name="constr_min_amount_" + self.components[i]
            )
            for i in range(len(self.components))
        ]

#### Mínimo de pratos principais

A restrição abaixo garante que uma quantidade mínima de pratos principais esteja presente nas refeições.

$\sum_{i=K+1}^{L}{x_{j}} \geq P$

In [ ]:
class DietSolver(DietSolver):  
    def add_main_dish_rep_constraints(self):
        return [
            self.model.add_constr(
                xsum([
                    self.model.var_by_name("x_" + dish)
                    for dish in self.main_dishes
                ])
                >=
                self.main_dish_min_rep,
                name="constr_min_main_dish"
            )
        ]

#### Mínimo de repetições de pratos culturais

Deve-se garantir que um número mínimo de cada prato cultural seja inserido no planejamento.

Garante-se que isso ocorra para pratos baseados em arroz:

<!-- $\sum_{j = 1}^{K^{arroz}}{x_{j}} \geq C^{min}$ -->
$x_{j} \geq C^{min}, j = 1 ... K^{arroz}$


Garante-se que isso ocorra para pratos baseados em feijão:

<!-- $\sum_{j = K^{arroz} + 1}^{K^{feijao}}{x_{j}} \geq C^{min}$ -->
$x_{j} \geq C^{min}, j = K^{arroz}+1 ... K^{feijao}$

In [ ]:
class DietSolver(DietSolver):  
    def add_cultural_dish_min_rep_constraints(self):
        constr = []
        for k, dishes in enumerate(self.cultural_dishes):
            constr +=  [
                self.model.add_constr(
                    self.model.var_by_name("x_" + dish)
                    >=
                    self.cultural_dish_min_rep,
                    name="constr_min_cultural_dish_" + str(k)
                )
                for dish in dishes
            ]
        return constr
        
        # return [
        #     self.model.add_constr(
        #         xsum([
        #             self.model.var_by_name("x_" + dish)
        #             for dish in dishes
        #         ])
        #         >=
        #         self.cultural_dish_min_rep,
        #         name="constr_min_cultural_dish_" + str(k)
        #     )
        #     for k, dishes in enumerate(self.cultural_dishes)
        # ]

#### Máximo de repetições de pratos culturais

Deve-se garantir que um número máximo de cada prato cultural não seja execedido.

Garante-se que isso ocorra para pratos baseados em arroz:

<!-- $\sum_{j = 1}^{K^{arroz}}{x_{j}} \leq C^{max}$ -->
$x_{j} \leq C^{max}, j = 1 ... K^{arroz}$

Garante-se que isso ocorra para pratos baseados em feijão:

<!-- $\sum_{j = K^{arroz} + 1}^{K^{feijao}}{x_{j}} \leq C^{max}$ -->
$x_{j} \leq C^{max}, j = K^{arroz}+1 ... K^{feijao}$


In [ ]:
class DietSolver(DietSolver):  
    def add_cultural_dish_max_rep_constraints(self):
        constr = []
        for k, dishes in enumerate(self.cultural_dishes):
            constr +=  [
                self.model.add_constr(
                    self.model.var_by_name("x_" + dish)
                    <=
                    self.cultural_dish_max_rep,
                    name="constr_max_cultural_dish_" + str(k)
                )
                for dish in dishes
            ]
        return constr

        # return [
        #     self.model.add_constr(
        #         xsum([
        #             self.model.var_by_name("x_" + dish)
        #             for dish in dishes
        #         ])
        #         <=
        #         self.cultural_dish_max_rep,
        #         name="constr_max_cultural_dish_" + str(k)
        #     )
        #     for k, dishes in enumerate(self.cultural_dishes)
        # ]

#### Repetição de pratos não culturais

As restrições abaixo garantem que pratos não culturais tenham um limite de repetição.

$x_{j} \leq O^{max}$, com $j = K+1 ... N$

In [ ]:
class DietSolver(DietSolver):  
    def add_other_dishes_max_rep(self):
        # Create a list without main and cultural dishes and 
        cultural_dishes_list = []
        for dishes in self.cultural_dishes:
            cultural_dishes_list += dishes
        
        other_dishes = list(
            set(self.dishes) 
            - 
            set(cultural_dishes_list)
        )

        return  [
            self.model.add_constr(
                self.model.var_by_name("x_" + dish)
                <=
                self.others_max_rep,
                name="constr_max_other_dish"
            )
            for dish in other_dishes
        ]

#### Domínio

As restrições abaixo definem o domínio das variáveis. Elas são implementadas na criação das variáveis

$x_{j} \geq 0, j = 1 ... N$

### Execução do exemplo

Abaixo encontra-se o código para a execução do exemplo apresentado. Será impresso o modelo matemático e a solução obtida.

In [ ]:
input_data = {
    "dishes" : dishes,
    "costs" : costs_per_person,
    "components" : components,
    "components_ingerdients_amount" : components_ingerdients_amount,
    "components_min_amount" : components_min_amount,
    "main_dishes" : main_dishes,
    "cultural_dishes" : cultural_dishes,
    "main_dish_min_rep" : main_dish_min_rep,
    "cultural_dish_min_rep" : cultural_dish_min_rep,
    "cultural_dish_max_rep" : cultural_dish_max_rep,
    "others_max_rep" : others_max_rep,
    "total_to_make" : total_to_make
}
model = DietSolver(input_data=input_data)
model.print_model()
model.optimize()
model.print_solution()